<a href="https://colab.research.google.com/github/timgluz/colab_notebooks/blob/master/SpacyCourse_chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data structures: Vocab, Lexemes, and StringStore

source: https://course.spacy.io/en/chapter2

In this lesson, we'll take a look at the shared vocabulary and how spaCy deals with strings.

spaCy stores all shared data in a vocabulary, the **Vocab**.

This includes words, but also the labels schemes for tags and entities.

Strings are only stored once in the `StringStore` via `nlp.vocab.strings`

In [ ]:
# install spacy

%pip install spacy

In [ ]:
# download medium size model package
%%python -m spacy download en_core_web_md

In [ ]:

# import the english language class
from spacy.lang.en import English

# create the nlp object
# contains the procesing pipeline,
# includes language-specific rules for tokenization
nlp = English()

In [ ]:
# Look up the string and hash in nlp.vocab.string
doc = nlp("I love coffee")
print("hash value:", nlp.vocab.strings["coffee"])
print("string value:", nlp.vocab.strings[3197928453018144401])

In [ ]:
# A Doc object also exposes its vocab and strings.

doc = nlp("I love coffee")
print("hash value:", doc.vocab.strings["coffee"])

### Lexemes

Lexemes are context-independent entries in the vocabulary.

You can get a lexeme by looking up a string or a hash ID in the vocab.

Lexemes expose attributes, just like tokens.

They hold context-independent information about a word, like the text, or whether the word consists of alphabetic characters.

Lexemes don't have part-of-speech tags, dependencies or entity labels. Those depend on the context.


In [ ]:
doc = nlp("I love coffee")
lexeme = nlp.vocab["coffee"]

# Print the lexical attributes
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

### The Doc object

The `Doc` is one of the central data structures in spaCy. It's created automatically when you process a text with the `nlp` object. But you can also instantiate the class manually.

After creating the `nlp` object, we can import the `Doc` class from `spacy.tokens`.


The spaces are a list of boolean values indicating whether the word is followed by a space. Every token includes that information – even the last one!

The Doc class takes three arguments: the shared vocab, the words and the spaces.


In [ ]:
# Create an nlp object
from spacy.lang.en import English
nlp = English()

# Import the Doc class
from spacy.tokens import Doc

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc[0])

### The Span object

A Span is a slice of a doc consisting of one or more tokens. The Span takes at least three arguments: the doc it refers to, and the start and end index of the span. Remember that the end index is exclusive!

In [ ]:
# Import the Doc and Span classes
from spacy.tokens import Doc, Span

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Create a span manually
span = Span(doc, 0, 2)

# Create a span with a label
span_with_label = Span(doc, 0, 2, label="GREETING")

# Add span to the doc.ents
doc.ents = [span_with_label]
print(span_with_label)

# Word vectors and semantic similarity

In this lesson, you'll learn how to use spaCy to predict how similar documents, spans or tokens are to each other.

The `Doc`, `Token` and `Span` objects have a `.similarity` method that takes another object and returns a floating point number between 0 and 1, indicating how similar they are.


**NB!** : In order to use similarity, you need a larger spaCy model that has word vectors included.



In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")
# Compare two documents
doc1 = nlp("I like fast food")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

# Compare a document with a token
doc = nlp("I like pizza")
token = nlp("soap")[0]

print(doc.similarity(token))

# Compare a span with a document
span = nlp("I like pizza and pasta")[2:5]
doc = nlp("McDonalds sells burgers")

print(span.similarity(doc))

In [ ]:
doc = nlp("I have a banana")
# Access the vector via the token.vector attribute
print(doc[3].vector)

# Combining models and rules

Combining statistical models with rule-based systems is one of the most powerful tricks you should have in your NLP toolbox.

![alt text](https://imgur.com/mULKyUal.png)

In [ ]:
import json
from spacy.lang.en import English

with open("exercises/en/countries.json") as f:
    COUNTRIES = json.loads(f.read())

nlp = English()
doc = nlp("Czech Republic may help Slovakia protect its airspace")

# Import the PhraseMatcher and initialize it
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Call the matcher on the test document and print the result
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])